<a href="https://colab.research.google.com/github/tiagosilveiraa/portifoliods/blob/main/Recomenda%C3%A7%C3%A3o_de_Livros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importação de bibliotecas

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

#1º - Importando Datasets

In [ ]:
%%capture
books =  pd.read_csv('https://drive.google.com/uc?export=download&id=18mBdTNm2dDu8Fhjng1G-b0WKr2ScXphF', sep=';', usecols=['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher'], encoding='latin-1', error_bad_lines=False)

In [ ]:
%%capture 
users = pd.read_csv('https://drive.google.com/uc?export=download&id=1ouljK77yUr1z89Y7CHKWXWFoZKVuT0E8', sep=';', encoding='latin-1', error_bad_lines=False)

In [ ]:
%%capture
ratings = pd.read_csv('https://drive.google.com/uc?export=download&id=1QeWWj7hoQxnRy6ooeTRllopdRNmplxwr', sep=';', encoding='latin-1', error_bad_lines=False)

#2º - Processamento de dados

In [ ]:
books = books.rename(str.lower, axis='columns').rename(columns={'book-title': 'title','book-author':'author', 'year-of-publication': 'year'})
users = users.rename(str.lower, axis='columns')
ratings = ratings.rename(str.lower, axis='columns').rename(columns={'book-rating': 'rating'})

In [ ]:
#Return de quantity of ratings per user
ratings['user-id'].value_counts()

11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: user-id, Length: 105283, dtype: int64

In [ ]:
#Only consider user that has made more than 200 ratings, this will garantee that the ratings are made with caution
x = ratings['user-id'].value_counts()> 200

In [ ]:
y = x[x].index
y

Int64Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352,
            110973, 235105,
            ...
             88793, 274808,  44296,  28634,  73681,  59727, 188951, 268622,
              9856, 155916],
           dtype='int64', length=899)

In [ ]:
y = x[x].index
print(y.shape)

(899,)


In [ ]:
ratings = ratings[ratings['user-id'].isin(y)]

In [ ]:
ratings_wb = ratings.merge(books, on='isbn')
ratings_wb.head()

,user-id,isbn,rating,title,author,year,publisher
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc


In [ ]:
number_rating = ratings_wb.groupby('title')['rating'].count().reset_index()

In [ ]:
number_rating.rename(columns={'rating': 'n_ratings'}, inplace=True)
number_rating

,title,n_ratings
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1
...,...,...
160275,Ã?Â?ber die Pflicht zum Ungehorsam gegen den S...,3
160276,Ã?Â?lpiraten.,1
160277,Ã?Â?rger mit Produkt X. Roman.,1
160278,Ã?Â?stlich der Berge.,1


In [ ]:
final_rating = ratings_wb.merge(number_rating, on = 'title')
final_rating

,user-id,isbn,rating,title,author,year,publisher,n_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
...,...,...,...,...,...,...,...,...
487680,275970,1892145022,0,Here Is New York,E. B. White,1999,Little Bookroom,1
487681,275970,1931868123,0,There's a Porcupine in My Outhouse: Misadventu...,Mike Tougias,2002,Capital Books (VA),1
487682,275970,3411086211,10,Die Biene.,Sybil GrÃ?Â¤fin SchÃ?Â¶nfeldt,1993,"Bibliographisches Institut, Mannheim",1
487683,275970,3829021860,0,The Penis Book,Joseph Cohen,1999,Konemann,1


In [ ]:
#Considerar apenas livros que tiveram mais de 50 avaliações
final_rating = final_rating[final_rating['n_ratings'] > 50]
final_rating.shape

(59903, 8)

In [ ]:
final_rating.drop_duplicates(['user-id', 'title'], inplace=True)
final_rating.shape

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


(57952, 8)

In [ ]:
final_rating

,user-id,isbn,rating,title,author,year,publisher,n_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
...,...,...,...,...,...,...,...,...
228880,268110,0743439651,0,Still Waters,Jennifer Lauck,2001,Atria,83
228881,127233,0061090956,0,Still Waters,Kerry Tucker,1992,Harper Mass Market Paperbacks (Mm),83
228882,262399,0061090956,0,Still Waters,Kerry Tucker,1992,Harper Mass Market Paperbacks (Mm),83
228883,178667,074350996X,0,Still Waters,Jennifer Lauck,2001,Simon &amp; Schuster Audio,83


#4º Pivotando as avaliações

In [ ]:
ratings_pv = final_rating.pivot_table(columns='user-id', index='title', values='rating', fill_value=0)

In [ ]:
ratings_pv

user-id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1st to Die: A Novel,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2nd Chance,0,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4 Blondes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
84 Charing Cross Road,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,10,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0,0,0,7,0,0,0,0,7,0,...,0,0,0,0,0,0,0,0,0,0
You Belong To Me,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#5º - Treinando o Modelo

In [ ]:
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(ratings_pv)

In [ ]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

#6º Aplicando o Modelo 


In [ ]:
distances, suggestions = model.kneighbors(ratings_pv.iloc[0,:].values.reshape(1,-1))
for i in range(len(suggestions)):
  print(ratings_pv.index[suggestions[i]])

Index(['1984', 'No Safe Place', 'A Civil Action', 'Foucault's Pendulum',
       'Long After Midnight'],
      dtype='object', name='title')
